In [275]:
from bs4 import BeautifulSoup
import requests
from bs4.element import Comment
import re
from nltk.tokenize import word_tokenize
import csv
import json
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pickle

In [276]:
base_url = 'https://en.wikipedia.org'

In [277]:
# Get the links in the paragraphs for wikipedia URL
def get_links_for_url(url):
    url_title = url[url.rfind('wiki')+5:]
    print("reading page: " + url_title)
    page = requests.get(url)
    try:
        soup = BeautifulSoup(page.text, "html5lib")
        tags = soup.find_all('p')
        res_links = []
        for tag in tags:
            links = tag.find_all('a', attrs={'href': re.compile("^/wiki/")})
            for link in links:
                res_links.append(link.get('href'))
        return res_links
    except Exception as e:
        print("Error loading page")
        return None
        

In [278]:
# Get cleaned text in the paragraph for wikipedia URL
def get_text_for_url(url):
    url_title = url[url.rfind('wiki')+5:]
    print("reading page: " + url_title)
    page = requests.get(url)
    try:
        soup = BeautifulSoup(page.text, "html5lib")
        text = soup.find_all('p')
        full_text = ""
        num = 0
        for t in text:
            if num > 5:
                break
            if not t.find('img') and (t.name == 'p' or t.name == 'ul'):
                text = str(t.getText().replace('\n', ''))
                full_text += re.sub("[\(\[].*?[\)\]]", "", text) + " "
                num += 1
        return full_text
    except Exception as e:
        print("Error loading page")
        return None

In [5]:
links = get_links_for_url(base_url + '/wiki/Computer_science')
links

reading page: Computer_science


['/wiki/Process_(engineering)',
 '/wiki/Data',
 '/wiki/Computer_program',
 '/wiki/Algorithm',
 '/wiki/Data_processing',
 '/wiki/Data_storage',
 '/wiki/Communication',
 '/wiki/Digital_data',
 '/wiki/Information',
 '/wiki/Computer_scientist',
 '/wiki/Theory_of_computation',
 '/wiki/Software_design',
 '/wiki/Software_system',
 '/wiki/Practical_disciplines',
 '/wiki/Computational_complexity_theory',
 '/wiki/Computer_graphics_(computer_science)',
 '/wiki/Programming_language_theory',
 '/wiki/Computer_programming',
 '/wiki/Programming_language',
 '/wiki/Complex_systems',
 '/wiki/Human%E2%80%93computer_interaction',
 '/wiki/Computer_accessibility',
 '/wiki/Digital_computer',
 '/wiki/Abacus',
 '/wiki/Algorithm',
 '/wiki/Wilhelm_Schickard',
 '/wiki/Mechanical_calculator',
 '/wiki/Gottfried_Leibniz',
 '/wiki/Stepped_Reckoner',
 '/wiki/Charles_Xavier_Thomas',
 '/wiki/Mechanical_calculator',
 '/wiki/Arithmometer',
 '/wiki/Charles_Babbage',
 '/wiki/Difference_Engine',
 '/wiki/Analytical_Engine',
 '

In [6]:
text = get_text_for_url(base_url + '/wiki/Computer_science')
text

reading page: Computer_science


' Computer science is the study of processes that interact with data and that can be represented as data in the form of programs. It enables the use of algorithms to manipulate, store, and communicate digital information. A computer scientist studies the theory of computation and the practice of designing software systems. Its fields can be divided into theoretical and practical disciplines. Computational complexity theory is highly abstract, while computer graphics emphasizes real-world applications. Programming language theory considers approaches to the description of computational processes, while computer programming itself involves the use of programming languages and complex systems. Human–computer interaction considers the challenges in making computers useful, usable, and accessible. The earliest foundations of what would become computer science predate the invention of the modern digital computer. Machines for calculating fixed numerical tasks such as the abacus have existed 

In [263]:
def reduce(u, v):
    return (u[0].union(v[0]), u[1] + v[1])

def clean_text(text):
    if text is None:
        return None
    regex = re.compile('[^a-zA-Z ]')
    text = regex.sub(' ', text)
    tokenized_text = word_tokenize(text.lower())
    return tokenized_text


def clean_text_for_vectorizer(text):
    if text is None:
        return None
    regex = re.compile('[^a-zA-Z ]')
    text = regex.sub(' ', text)
    return text.lower()


In [248]:
def generate_representation_for_course_description(course_description, branching_factor, search_depth, representation):
    '''
    course_description: [str], string of text description for a course
    branching_factor: float, in range(0, 1), what percentage of the links we want to traverse at each step
    search_depth: the maximum depth of link traversals starting from an original word in the course_description
    
    Returns: (links, representation)
    '''
    course_description
    res = (set(), "")
    explored_urls = set()
    for word in course_description:
        url = '/wiki/' + word
        res = reduce(res, generate_representation_for_word(explored_urls, url, branching_factor, search_depth, ""))
    return res
    
    

def generate_representation_for_word(explored_urls, url, branching_factor, search_depth, representation):
    '''
    explored_urls: set, of urls that have already been explored
    url: the url we wish to find the wikipedia article on, 
    if word is multiple words, it will appear as /wiki/computer_science, /wiki/computer_graphics, etc
    branching_factor: float, in range(0, 1), which percentage of the links we want to traverse at each step
    search_depth: int, the maximum depth of link traversals starting from word
    
    return: (links, represenation)
    links: list of strings
    representation: string
    '''
    if search_depth == 0 or url in explored_urls:
        return (set(), "")
    explored_urls.add(url)
    links = get_links_for_url(base_url + url)
    if links is None: # Only if the url is not valid
        return (explored_urls, representation)
    text = get_text_for_url(base_url + url)
    representation += " " + text
    next_links = links[:min(8, int(branching_factor * len(links)))]
    
    res = (explored_urls, representation)
    for link in next_links:
        res = reduce(res, generate_representation_for_word(explored_urls, link, branching_factor, search_depth - 1, ""))
        
    return res

In [215]:
rep_for_cs_course = generate_representation_for_course_description(["Computer_science", "biology"], 0.05, 2, "")

reading page: Computer_science
reading page: Computer_science
reading page: Process_(engineering)
reading page: Process_(engineering)
reading page: Data
reading page: Data
reading page: Computer_program
reading page: Computer_program
reading page: Algorithm
reading page: Algorithm
reading page: Data_processing
reading page: Data_processing
reading page: Data_storage
reading page: Data_storage
reading page: Communication
reading page: Communication
reading page: Digital_data
reading page: Digital_data
reading page: biology
reading page: biology
reading page: Natural_science
reading page: Natural_science
reading page: Life
reading page: Life
reading page: Organism
reading page: Organism
reading page: Anatomy
reading page: Anatomy
reading page: Biochemistry
reading page: Biochemistry
reading page: Molecular_biology
reading page: Molecular_biology
reading page: Physiology
reading page: Physiology
reading page: Developmental_biology
reading page: Developmental_biology


In [216]:
rep_for_cs_course

({'/wiki/Algorithm',
  '/wiki/Anatomy',
  '/wiki/Biochemistry',
  '/wiki/Communication',
  '/wiki/Computer_program',
  '/wiki/Computer_science',
  '/wiki/Data',
  '/wiki/Data_processing',
  '/wiki/Data_storage',
  '/wiki/Developmental_biology',
  '/wiki/Digital_data',
  '/wiki/Life',
  '/wiki/Molecular_biology',
  '/wiki/Natural_science',
  '/wiki/Organism',
  '/wiki/Physiology',
  '/wiki/Process_(engineering)',
  '/wiki/biology'},
 '  Computer science is the study of processes that interact with data and that can be represented as data in the form of programs. It enables the use of algorithms to manipulate, store, and communicate digital information. A computer scientist studies the theory of computation and the practice of designing software systems. Its fields can be divided into theoretical and practical disciplines. Computational complexity theory is highly abstract, while computer graphics emphasizes real-world applications. Programming language theory considers approaches to the

In [73]:
#rep_for_Computer_science = generate_representation_for_word(set(), '/wiki/Computer_science', 0.05, 2, "")
# rep_for_Computer_science

In [217]:
with open("./data/courseroster/full_json.txt") as f:
    cornell_course_descriptions = json.load(f)

In [218]:
cornell_course_descriptions.keys()

dict_keys(['VIEN', 'BSOC', 'PLPPM', 'FREN', 'GREEK', 'PMA', 'AIIS', 'KOREA', 'NCC', 'CHEM', 'TURK', 'TAMIL', 'YORUB', 'MAE', 'BENGL', 'PLBIO', 'VISST', 'COMM', 'MEDVL', 'ECON', 'ENMGT', 'CHIN', 'HIST', 'PERSN', 'PUNJB', 'ORIE', 'NEPAL', 'ILRLR', 'URDU', 'BEE', 'EDUC', 'ENGRI', 'WOLOF', 'BIOG', 'HD', 'INFO', 'PSYCH', 'ENGRD', 'ARAB', 'STSCI', 'SNLIT', 'PHYS', 'PORT', 'RELST', 'ALS', 'FSAD', 'PLSCS', 'COGST', 'CZECH', 'MATH', 'NBA', 'PLHRT', 'ENTOM', 'GERST', 'JWST', 'ILRLE', 'TIBET', 'MSE', 'VETMI', 'IARD', 'PADM', 'ECE', 'BIOMI', 'TOX', 'AKKAD', 'SINHA', 'ARKEO', 'NTRES', 'SPAN', 'COLLS', 'KHMER', 'AMST', 'AEM', 'JPLIT', 'CHLIT', 'HEBRW', 'BIOMS', 'PLSCI', 'ROMS', 'ROMAN', 'PHIL', 'BIONB', 'ASTRO', 'FGSS', 'DSOC', 'BIOEE', 'ILRHR', 'VETMM', 'CAPS', 'PE', 'NACCT', 'AAS', 'ASRC', 'STS', 'JAPAN', 'MOGRK', 'KANAD', 'AIRS', 'ILROB', 'ENGRC', 'ENGL', 'HINDI', 'VTPMD', 'TECH', 'ILRIC', 'LATIN', 'SWAHL', 'BME', 'VTBMS', 'ART', 'NS', 'HADM', 'ENGRG', 'VTMED', 'ZULU', 'GRAD', 'ANSC', 'LA', 'COML

In [219]:
cornell_course_descriptions['CS'][0]

{'courseNumber': '1110',
 'courseTitle': 'Introduction to Computing Using Python',
 'description': 'Programming and problem solving using Python. Emphasizes principles of software development, style, and testing. Topics include procedures and functions, iteration, recusion, arrays and vectors, strings, an operational model of procedure and function calls, algorithms, exceptions, object-oriented programming, and GUIs (graphical user interfaces). Weekly labs provide guided practice on the computer, with staff present to help. Assignments use graphics and GUIs to help develop fluency and understanding.',
 'offered': 'Fall, Spring, Summer.',
 'outcomes': ' Be fluent in the use of procedural statements -assignments, conditional statements, loops, method calls- and  arrays. Be able to design, code, and test small Python programs that meet requirements expressed in English. This includes a basic understanding of top-down design. Understand the concepts of object-oriented programming as used i

In [207]:
corpus = []
all_major_courses = [cornell_course_descriptions[key] for key in cornell_course_descriptions.keys()]
all_courses = []
for major_courses in all_major_courses:
    all_courses += major_courses

for course_data in all_courses:
    course_number = course_data['courseNumber']
    course_title = course_data['courseTitle']
    course_desc = course_data['description']
    cleaned_course_desc = clean_text_for_vectorizer(course_desc)
    if cleaned_course_desc != None and cleaned_course_desc != []: 
        corpus.append(cleaned_course_desc)
    course_outcome = None
    try:
        course_outcome = course_data['outcome']
    except Exception as e:
        course_outcome = ""

In [249]:
print(len(corpus))
print(corpus[4000])

4089
some of our very best evidence for roman art survives in the form of frescoes in rome ostia and the bay of naples exploring imperial palaces rural villas town houses shops baths tombs taverns and gardens we will examine the visual dynamics and sociocultural significance of wallpaintings within their original archaeological contexts the study of frescoes offers an exciting means of tackling important questions relating to roman social history alongside the complexities of representation within the grecoroman visual tradition including the relationship between art and nature the use of myth the spatial dynamics of interior decorative schemes visualverbal relations and concepts of ornament medium and abstraction this course will be taught as a traveling seminar which will include a trip to rome and bay of naples over spring break spaces are limited


In [209]:
vectorizer = TfidfVectorizer(lowercase= True, preprocessor=lambda x: x, stop_words='english', max_df=0.8, min_df = 2,smooth_idf=True)
X = vectorizer.fit_transform(corpus)
doc_by_vocab = vectorizer.fit_transform([d for d in corpus]).toarray()

In [223]:
def get_top_n_tfidf_terms_from_text(vectorizer, text, n):
    print("S")
    if text is None:
        return []
    feature_array = np.array(vectorizer.get_feature_names())
    transformed_text = vectorizer.transform([text])
    tfidf_sorting = np.argsort(transformed_text.toarray()).flatten()[::-1]
    k = len(text.split())
    top_n = feature_array[tfidf_sorting][:min(k, n)]
    return top_n

def get_top_n_important_terms_from_text(vectorizer, text, n):
    pass
    

In [264]:
cs4110 = "Introduction to the design of systems programs, with emphasis on multiprogrammed operating systems. Topics include concurrency, synchronization, deadlocks, memory management, protection, input-output methods, networking, file systems and security. The impact of network and distributed computing environments on operating systems is also discussed."
print(get_top_n_tfidf_terms_from_text(vectorizer, clean_text_for_vectorizer(cs4110), 5))

S
['systems' 'operating' 'file' 'concurrency' 'synchronization']


In [279]:
eng_majors = ['BEE', 'BME', 'CHEME', 'CEE', 'CS', 'EAS','ECE', 'AEP', 'BEE', 'INFO', 'MSE', 'MAE', 'ORIE', 'ENGRC', 'ENGRD', 'ENGRG', 'ENGRI', 'ENMGT', 'STSCI', 'SYSEN']
eng_courses = [[cornell_course_descriptions[major_code] for major_code in eng_majors]]
all_courses = []
for major_code in eng_majors:
    for course_data in cornell_course_descriptions[major_code]:
        all_courses.append(course_data)
print(len(all_courses))

736


In [280]:
course_representations = {}
for course_data in all_courses:
    course_number = course_data['courseNumber']
    dept = course_data['subject']
    course_desc = course_data['description']
    cleaned_course_desc = clean_text_for_vectorizer(course_desc)
    print(course_number)
    print(course_desc)
    print(cleaned_course_desc)
    top_tfidf_words = get_top_n_tfidf_terms_from_text(vectorizer, cleaned_course_desc, 10)
    print(top_tfidf_words)
    course_representation = ' '.join(top_tfidf_words)
    course_representations[dept + ' ' +str(course_number)] = course_representation
    print(course_representation)
    print("************************")

1200
Forum covering the career opportunities for engineering students and the activities and curricula that lead to these opportunities. A series of seminars are given by practicing engineers, Cornell faculty members, alumni, staff from Cornell career services offices, and students. Students develop their undergraduate course plans, complete a web search assignment to locate jobs and internships, learn to craft a resume and cover letter and explore opportunities for life long learning.
forum covering the career opportunities for engineering students and the activities and curricula that lead to these opportunities  a series of seminars are given by practicing engineers  cornell faculty members  alumni  staff from cornell career services offices  and students  students develop their undergraduate course plans  complete a web search assignment to locate jobs and internships  learn to craft a resume and cover letter and explore opportunities for life long learning 
S
['opportunities' 'car

energy renewable scale project small drying quantifying spreadsheets biomass systems
************************
4200
Natural particles such as clay minerals, mineral oxides, and organo-mineral composites facilitate the cycling of elements in the environment, transport and degradation of contaminants, and physico-chemical processes in environmental and chemical engineering. This course will cover the surface chemistry of these particles relevant to their role in these natural and engineered processes. The applications of chemical kinetics, chemical equilibrium, and molecular spectroscopy in particle characterization will be discussed.
natural particles such as clay minerals  mineral oxides  and organo mineral composites facilitate the cycling of elements in the environment  transport and degradation of contaminants  and physico chemical processes in environmental and chemical engineering  this course will cover the surface chemistry of these particles relevant to their role in these natur

************************
5952
Comprehensive engineering design projects relating to the candidate's area of specialization. Projects are supervised by a BEE faculty member on an individual basis. A formal project report and oral presentation of the design project are required for completion of the course(s). A minimum of 3 to a maximum of 9 credits of BEE 5951-5952 is required for the M.Eng. degree.
comprehensive engineering design projects relating to the candidate s area of specialization  projects are supervised by a bee faculty member on an individual basis  a formal project report and oral presentation of the design project are required for completion of the course s   a minimum of   to a maximum of   credits of bee           is required for the m eng  degree 
S
['bee' 'required' 'projects' 'specialization' 'project' 'design' 'maximum'
 'minimum' 'relating' 'degree']
bee required projects specialization project design maximum minimum relating degree
************************
6200
N

 'tissue' 'analysis' 'considerations' 'depth']
incorporating cellular biomaterial biomaterials length design tissue analysis considerations depth
************************
5910
Design and economic evaluation of a biomedical engineering device or therapeutic strategy. Team projects are encouraged.
design and economic evaluation of a biomedical engineering device or therapeutic strategy  team projects are encouraged 
S
['device' 'therapeutic' 'encouraged' 'biomedical' 'strategy' 'team'
 'evaluation' 'engineering' 'projects' 'economic']
device therapeutic encouraged biomedical strategy team evaluation engineering projects economic
************************
5911
None
None
S
[]

************************
5920
Once a proposal for the project has been approved by the Sponsor or Faculty member, the student must ("deliver on time and under budget") produce a tangible work product. Scheduling activities, ordering supplies, assembling, testing the device or procedure, and documenting the work and ou

['image' 'processing' 'spectral' 'enhancement' 'introduction' 'remote'
 'extraction' 'graphics' 'sensing' 'classification']
image processing spectral enhancement introduction remote extraction graphics sensing classification
************************
6200
Development and application of deterministic and stochastic optimization and simulation models for aiding in water-resources planning and management. Covers river-basin modeling, including water allocation to multiple purposes, reservoir design and operation, irrigation planning and operation, hydropower-capacity development, flow augmentation, flood control and protection, and urban water supply portfolio management.
development and application of deterministic and stochastic optimization and simulation models for aiding in water resources planning and management  covers river basin modeling  including water allocation to multiple purposes  reservoir design and operation  irrigation planning and operation  hydropower capacity developm

S
['games' 'automation' 'moocs' 'misconceptions' 'scaffolding'
 'instructional' 'learning' 'massive' 'release' 'experimentation']
games automation moocs misconceptions scaffolding instructional learning massive release experimentation
************************
6700
Covers a variety of areas in AI, including knowledge representation, automated reasoning, learning, game-playing, and planning, with an emphasis on computational issues.
covers a variety of areas in ai  including knowledge representation  automated reasoning  learning  game playing  and planning  with an emphasis on computational issues 
S
['ai' 'automated' 'playing' 'reasoning' 'game' 'computational'
 'representation' 'areas' 'planning' 'variety']
ai automated playing reasoning game computational representation areas planning variety
************************
6740
Graduate-level introduction to technologies for the computational treatment of information in human-language form, covering modern natural-language processing (NLP)

S
['hydrogen' 'operator' 'atom' 'identical' 'spin' 'angular' 'perturbation'
 'radiation' 'momentum' 'equation']
hydrogen operator atom identical spin angular perturbation radiation momentum equation
************************
3630
Practical electronics as encountered in a scientific or engineering research/development environment. Analyze, design, build, and test circuits using discrete components and integrated circuits. Analog circuits: resistors, capacitors, operational amplifiers, feedback amplifiers, oscillators, comparators, passive and active filters, diodes, and transistor switches and amplifiers. Digital circuits: combinational and sequential logic (gates, flipflops, registers, counters, timers), analog to digital (ADC) and digital to analog (DAC) conversion, signal averaging, and computer architecture and interfacing. Additional topics may include analog and digital signal processing, light wave communications, transducers, noise reduction techniques, and computer-aided circuit

['project' 'sponsor' 'arrangements' 'led' 'spend' 'hour' 'expected'
 'hours' 'team' 'week']
project sponsor arrangements led spend hour expected hours team week
************************
5100
This course will introduce students to working with data in the context of modern web applications. These include data representation with relational and non-relational databases, data mining to find patterns and make predictions, and graphical presentation for visualization.
this course will introduce students to working with data in the context of modern web applications  these include data representation with relational and non relational databases  data mining to find patterns and make predictions  and graphical presentation for visualization 
S
['relational' 'data' 'graphical' 'mining' 'predictions' 'databases'
 'visualization' 'web' 'patterns' 'representation']
relational data graphical mining predictions databases visualization web patterns representation
************************
5200
None
N

aerospace guidance mechanical area member engineering independent faculty research failures
************************
1380
For description, see CS 1380.
for description  see cs      
S
['cs' 'description' 'fail' 'fake']
cs description fail fake
************************
3120
The practical use of software tools and mathematical methods from operations research, machine learning, statistics and data science. Software tools include structured query language (SQL), geographical information systems (GIS), Excel and Visual Basic programming (VBA), and programming in a scripting language (either R or Python). Operations research methods include inventory management, discrete event simulation, and an introduction to the analysis of queuing systems. Machine learning and statistical methods include multiple linear regression, classification, logistic regression, clustering, time-series forecasting, and the design and analysis of A/B tests. These topics will be presented in the context of business 

In [281]:
course_representations['INFO 4300']

'social information retrieving unstructured retrieval vast appears amounts textual analysis'

In [282]:
pickle.dump(course_representations, open("eng_representations_just_descriptions_2.p", "wb" ))

In [241]:
course_representations_simple = {}
for course_data in cornell_course_descriptions['CS']:
    course_number = course_data['courseNumber']
    course_desc = course_data['description']
    cleaned_course_desc = clean_text_for_vectorizer(course_desc)
    print(course_number)
    print(course_desc)
    print(cleaned_course_desc)
    top_tfidf_words = get_top_n_tfidf_terms_from_text(vectorizer, cleaned_course_desc, 5)
    print(top_tfidf_words)
    course_representation = generate_representation_for_course_description(top_tfidf_words, 0.10, 1, "")
    course_representations_simple['CS '+str(course_number)] = course_representation
    print(course_representation)
    print("************************")

1110
Programming and problem solving using Python. Emphasizes principles of software development, style, and testing. Topics include procedures and functions, iteration, recusion, arrays and vectors, strings, an operational model of procedure and function calls, algorithms, exceptions, object-oriented programming, and GUIs (graphical user interfaces). Weekly labs provide guided practice on the computer, with staff present to help. Assignments use graphics and GUIs to help develop fluency and understanding.
programming and problem solving using python emphasizes principles of software development style and testing topics include procedures and functions iteration recusion arrays and vectors strings an operational model of procedure and function calls algorithms exceptions objectoriented programming and guis graphical user interfaces weekly labs provide guided practice on the computer with staff present to help assignments use graphics and guis to help develop fluency and understanding
S

reading page: matlab
reading page: programming
reading page: programming
reading page: conditionals
reading page: conditionals
reading page: vectorized
reading page: vectorized
reading page: constructs
reading page: constructs
({'/wiki/programming', '/wiki/constructs', '/wiki/conditionals', '/wiki/vectorized', '/wiki/matlab'}, "  MATLAB  is a multi-paradigm numerical computing environment and proprietary programming language developed by MathWorks. MATLAB allows matrix manipulations, plotting of functions and data, implementation of algorithms, creation of user interfaces, and interfacing with programs written in other languages, including C, C++, C#, Java, Fortran and Python. Although MATLAB is intended primarily for numerical computing, an optional toolbox uses the MuPAD symbolic engine, allowing access to symbolic computing abilities. An additional package, Simulink, adds graphical multi-domain simulation and model-based design for dynamic and embedded systems. As of 2018, MATLAB ha

reading page: science
reading page: mind
reading page: mind
reading page: like
reading page: like
reading page: disciplines
reading page: disciplines
reading page: intities
reading page: intities
({'/wiki/disciplines', '/wiki/intities', '/wiki/science', '/wiki/mind', '/wiki/like'}, '   Science  is a systematic enterprise that builds and organizes knowledge in the form of testable explanations and predictions about the universe. The earliest roots of science can be traced to Ancient Egypt and Mesopotamia in around 3500 to 3000 BCE. Their contributions to mathematics, astronomy, and medicine entered and shaped Greek natural philosophy of classical antiquity, whereby formal attempts were made to explain events of the physical world based on natural causes. After the fall of the Western Roman Empire, knowledge of Greek conceptions of the world deteriorated in Western Europe during the early centuries  of the Middle Ages but was preserved in the Muslim world during the Islamic Golden Age. T

reading page: shell
reading page: computational
reading page: computational
reading page: files
reading page: files
reading page: installations
reading page: installations
reading page: strings
reading page: strings
({'/wiki/shell', '/wiki/computational', '/wiki/installations', '/wiki/strings', '/wiki/files'}, ' Shell may refer to:  Computation is any type of calculation that includes both arithmetical and non-arithmetical steps and follows a well-defined model, for example an algorithm. The study of computation is paramount to the discipline of computer science. A computation can be seen as a purely physical phenomenon occurring inside a closed physical system called a computer.Examples of such physical systems include digital computers, mechanical computers, quantum computers, DNA computers, molecular computers, microfluidics-based computers, analog computers, or wetware computers.This point of view has been adopted by the physics of computation, a branch of theoretical physics, as w

reading page: php
reading page: database
reading page: database
reading page: client
reading page: client
reading page: web
reading page: web
reading page: creation
reading page: creation
({'/wiki/web', '/wiki/client', '/wiki/php', '/wiki/creation', '/wiki/database'}, '  since version 7.0: PHP: Hypertext Preprocessor  is a general-purpose programming language originally designed for web development. It was originally created by Rasmus Lerdorf in 1994; the PHP reference implementation is now produced by The PHP Group. PHP originally stood for Personal Home Page, but it now stands for the recursive initialism PHP: Hypertext Preprocessor. PHP code may be executed with a command line interface , embedded into HTML code, or it can be used in combination with various web template systems, web content management systems, and web frameworks. PHP code is usually processed by a PHP interpreter implemented as a module in a web server or as a Common Gateway Interface  executable. The web server co

...

reading page: graphics
reading page: vision
reading page: vision
reading page: computer
reading page: computer
reading page: research
reading page: research
reading page: foster
reading page: foster
({'/wiki/computer', '/wiki/graphics', '/wiki/research', '/wiki/vision', '/wiki/foster'}, ' Graphics  are visual images or designs on some surface, such as a wall, canvas, screen, paper, or stone to inform, illustrate, or entertain. In contemporary usage, it includes a pictorial representation of data, as in computer-aided design and manufacture, in typesetting and the graphic arts, and in educational and recreational software. Images that are generated by a computer are called computer graphics. Examples are photographs, drawings, Line art, graphs, diagrams, typography, numbers, symbols, geometric designs, maps, engineering drawings,or other images. Graphics often combine text, illustration, and color. Graphic design may consist of the deliberate selection, creation, or arrangement of typog

reading page: ai
reading page: workinprogress
reading page: workinprogress
reading page: emphasizing
reading page: emphasizing
reading page: researchers
reading page: researchers
reading page: results
reading page: results
({'/wiki/workinprogress', '/wiki/researchers', '/wiki/results', '/wiki/emphasizing', '/wiki/ai'}, ' AI, A.I., Ai, or ai may refer to:  Other reasons this message may be displayed:  Other reasons this message may be displayed:   Research comprises "creative and systematic work undertaken to increase the stock of knowledge, including knowledge of humans, culture and society, and the use of this stock of knowledge to devise new applications." It is used to establish or confirm facts, reaffirm the results of previous work, solve new or existing problems, support theorems, or develop new theories. A research project may also be an expansion on past work in the field. Research projects can be used to develop further knowledge on a topic, or in the example of a school resea

reading page: nlp
reading page: soon
reading page: soon
reading page: simply
reading page: simply
reading page: mandatory
reading page: mandatory
reading page: leads
reading page: leads
({'/wiki/nlp', '/wiki/soon', '/wiki/mandatory', '/wiki/simply', '/wiki/leads'}, ' NLP may refer to:  Soon may refer to:  Simply may refer to:  Mandate may refer to:  In typography, leading  refers to the distance between adjacent lines of type; however, the exact definition has become confused. In the days of hand-typesetting, it referred to the thin strips of lead that were inserted into the forms to increase the vertical distance between lines of type; in this case, the leading would be defined as the difference between 2 quantities: the size of the type and the distance from one baseline to the next. For instance, given a type size of 10\xa0points and a distance between baselines of 12\xa0points, the leading would be 2\xa0points; put another way, a leading of 2\xa0points means there is a distance of 

reading page: cryptography
reading page: discussing
reading page: discussing
reading page: classical
reading page: classical
reading page: papers
reading page: papers
reading page: recent
reading page: recent
({'/wiki/discussing', '/wiki/classical', '/wiki/recent', '/wiki/cryptography', '/wiki/papers'}, '  Cryptography or cryptology ) is the practice and study of techniques for secure communication in the presence of third parties called adversaries. More generally, cryptography is about constructing and analyzing protocols that prevent third parties or the public from reading private messages; various aspects in information security such as data confidentiality, data integrity, authentication, and non-repudiation are central to modern cryptography. Modern cryptography exists at the intersection of the disciplines of mathematics, computer science, electrical engineering, communication science, and physics. Applications of cryptography include electronic commerce, chip-based payment car

reading page: doctoral
reading page: research
reading page: research
({'/wiki/research', '/wiki/doctoral'}, ' A doctorate  or doctor\'s degree  or doctoral degree, is an academic degree awarded by universities, derived from the ancient formalism licentia docendi   In most countries, it is a research degree that qualifies the holder to teach at university level in the degree\'s field, or to work in a specific profession. There are a variety of names for doctoral degrees; the most common is the Doctor of Philosophy , which is awarded in many different fields, ranging from the humanities to scientific disciplines. In the United States and some other countries, there are also some types of vocational, technical, or professional degrees that are referred to as doctorates. Professional doctorates historically came about to meet the needs of practitioners in a variety of disciplines. However, the necessity of these degrees may vary greatly across disciplines, making their significance unclear

In [242]:
pickle.dump( course_representations_simple, open( "cs_representations_simple.p", "wb" ) )